### Import relevant libraries

In [1]:
import pandas as pd
import numpy as np

### Import data and reorder columns

In [2]:
# Read in df
df = pd.read_csv('randstad.csv')

# Reorder columns
lst_col = ['job_name', 'experience', 'category', 'job_type', 'salary','location', 'first_paragraph',
           'description', 'ref_number', 'posting_date', 'scrape_datetime']
df = df[lst_col]

df.head(3)

,job_name,experience,category,job_type,salary,location,first_paragraph,description,ref_number,posting_date,scrape_datetime
0,ForkLift Operator,Experienced,wholesaler of hardware,Temp to Perm,$17 per hour,"Rancho Cucamonga, CA",Are you looking for a full-time/long term oppo...,Are you looking for a full-time/long term oppo...,AB_3455910,"Thursday, September 3, 2020",2020-09-07 16:07:40.177048
1,Materiel Handler,Experienced,parse_error,Temporary,$12 - $14 per hour,"Des Moines, IA",Are you looking to join a company that can pro...,Are you looking to join a company that can pro...,AB_3455835,"Thursday, September 3, 2020",2020-09-07 16:07:40.343085
2,Application Development Manager,parse_error,Manufacturing,Permanent,"$90,000 - $110,000 per year","Phoenix, AZ",job summary:,job summary:\n\nAre you looking for your next ...,797532,"Thursday, September 3, 2020",2020-09-07 16:07:40.870705


### Text Manipulation

In [3]:
# Replace the parse_error with NA
df = df.replace('parse_error', np.nan).replace(' ', np.nan).replace('', np.nan)

# Determine whether salary is annual or hourly
df['salary_type'] = df['salary'].str.find('year')
df['salary_type'] = np.where(df['salary_type']!=-1, 'annual', 'hourly')

# Remove non-needed characters from salary
def salary_formatter(s):
    return ''.join([i for i in str(s) if i.isnumeric() or i in ['.', '-'] ])
df['salary'] = df['salary'].apply(salary_formatter)

# Make new columns for low and high end salary 
df_salary_range = df["salary"].str.split("-", n = 1, expand = True)
df.insert(4, 'salary_low', df_salary_range[0])
df['salary_low'] = df['salary_low'].replace('', np.nan).astype(float)
df.insert(5, 'salary_high', df_salary_range[1])
df['salary_high'] = df['salary_high'].replace('', np.nan).astype(float)

# Recalculate salary
df['temp_code'] = df['salary_low'].isnull().map({True: 0, False: 1})
df['temp_code'] = df['temp_code'] + df['salary_high'].isnull().map({True: 0, False: 1})
df['salary'] = np.where(df['temp_code']==1, df['salary_low'], (df['salary_low'] + df['salary_high']) / 2)
del df['temp_code']

# Make new columns for city and state 
df_location = df["location"].str.split(",", n = 1, expand = True)
df.insert(7, 'city', df_location[0])
df['city'] = df['city'].replace('', np.nan)
df.insert(8, 'state', df_location[1])
df['state'] = df['state'].replace('', np.nan)
del df['location']

# Convert datetime
df['scrape_datetime'] = pd.to_datetime(df['scrape_datetime'], infer_datetime_format=True)  

df.head(3)

,job_name,experience,category,job_type,salary_low,salary_high,salary,city,state,first_paragraph,description,ref_number,posting_date,scrape_datetime,salary_type
0,ForkLift Operator,Experienced,wholesaler of hardware,Temp to Perm,17.0,NaN,17.0,Rancho Cucamonga,CA,Are you looking for a full-time/long term oppo...,Are you looking for a full-time/long term oppo...,AB_3455910,"Thursday, September 3, 2020",2020-09-07 16:07:40.177048,hourly
1,Materiel Handler,Experienced,NaN,Temporary,12.0,14.0,13.0,Des Moines,IA,Are you looking to join a company that can pro...,Are you looking to join a company that can pro...,AB_3455835,"Thursday, September 3, 2020",2020-09-07 16:07:40.343085,hourly
2,Application Development Manager,NaN,Manufacturing,Permanent,90000.0,110000.0,100000.0,Phoenix,AZ,job summary:,job summary:\n\nAre you looking for your next ...,797532,"Thursday, September 3, 2020",2020-09-07 16:07:40.870705,annual


### Change format of states

In [4]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))


In [5]:
df['state'] = df['state'].str.strip().map(abbrev_us_state)
df.head(3)

,job_name,experience,category,job_type,salary_low,salary_high,salary,city,state,first_paragraph,description,ref_number,posting_date,scrape_datetime,salary_type
0,ForkLift Operator,Experienced,wholesaler of hardware,Temp to Perm,17.0,NaN,17.0,Rancho Cucamonga,California,Are you looking for a full-time/long term oppo...,Are you looking for a full-time/long term oppo...,AB_3455910,"Thursday, September 3, 2020",2020-09-07 16:07:40.177048,hourly
1,Materiel Handler,Experienced,NaN,Temporary,12.0,14.0,13.0,Des Moines,Iowa,Are you looking to join a company that can pro...,Are you looking to join a company that can pro...,AB_3455835,"Thursday, September 3, 2020",2020-09-07 16:07:40.343085,hourly
2,Application Development Manager,NaN,Manufacturing,Permanent,90000.0,110000.0,100000.0,Phoenix,Arizona,job summary:,job summary:\n\nAre you looking for your next ...,797532,"Thursday, September 3, 2020",2020-09-07 16:07:40.870705,annual


In [6]:
df.head(3)

,job_name,experience,category,job_type,salary_low,salary_high,salary,city,state,first_paragraph,description,ref_number,posting_date,scrape_datetime,salary_type
0,ForkLift Operator,Experienced,wholesaler of hardware,Temp to Perm,17.0,NaN,17.0,Rancho Cucamonga,California,Are you looking for a full-time/long term oppo...,Are you looking for a full-time/long term oppo...,AB_3455910,"Thursday, September 3, 2020",2020-09-07 16:07:40.177048,hourly
1,Materiel Handler,Experienced,NaN,Temporary,12.0,14.0,13.0,Des Moines,Iowa,Are you looking to join a company that can pro...,Are you looking to join a company that can pro...,AB_3455835,"Thursday, September 3, 2020",2020-09-07 16:07:40.343085,hourly
2,Application Development Manager,NaN,Manufacturing,Permanent,90000.0,110000.0,100000.0,Phoenix,Arizona,job summary:,job summary:\n\nAre you looking for your next ...,797532,"Thursday, September 3, 2020",2020-09-07 16:07:40.870705,annual


### Write to csv

In [7]:
df.to_csv('randstad_post_processed.csv', index=False)